In [ ]:
#10 repeats of outer_cv
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)   
nb_repeat = 10
repeat_y_true = []
repeat_predictions = []

for i in range(nb_repeat):
    cv = 0
    cv_predictions=[]
    cv_y_true= []
    for train_index, test_index in kf.split(X,y):

        X_train, X_test = X[train_index], X[test_index] 
        y_train, y_test = y[train_index], y[test_index]

        weights_file='/weights/' + model.name + str(nb_repeat) + '_' + str(nb_cv) +'_best_pretrained_weights.npy'
        model.load_weights(weights_file)
        prediction = model.predict(X_test)
        cv_predictions.append(prediction)
        cv_y_true.append(y_test)
        cv+=1
    repeat_predictions.append(cv_predictions)
    repeat_y_true.append(cv_y_true)

In [ ]:
def print_output(repeat_predictions,repeat_y_true):
    repeat_y_pred = []
    repeat_y_pred_proba = []
    repeat_matrix = []
    repeat_accuracy = []
    repeat_sensitivity = []
    repeat_specificity = []
    
    for i in range(len(repeat_predictions)):
        cv_y_pred = []
        cv_y_pred_proba = []
        cv_matrix = []
        cv_accuracy = []
        cv_sensitivity = []
        cv_specificity = []
        for j in range(len(repeat_predictions[i])):
            predictions = repeat_predictions[i][j]
            y_pred = (predictions > 0.5).argmax(axis=1)
            y_pred_proba=predictions[:, 1]
            y_true = repeat_y_true[i][j]
            
            #confusion matrix
            matrix = metrics.confusion_matrix(y_true, y_pred) 
            #accuracy
            total = sum(sum(matrix))
            accuracy = (matrix[0, 0] + matrix[1, 1]) / total

            #sensitivity & specificity
            specificity=matrix[0, 0] / (matrix[0, 0] + matrix[0, 1])
            sensitivity=matrix[1, 1] / (matrix[1, 0] + matrix[1, 1])
            print(accuracy,sensitivity,specificity)
            
            cv_y_pred.append(y_pred)
            cv_y_pred_proba.append(y_pred_proba)
            cv_matrix.append(matrix)
            cv_accuracy.append(accuracy)
            cv_sensitivity.append(sensitivity)
            cv_specificity.append(specificity)
            
        repeat_y_pred.append(cv_y_pred)
        repeat_y_pred_proba.append(cv_y_pred_proba)
        repeat_matrix.append(cv_matrix)
        repeat_accuracy.append(cv_accuracy)
        repeat_sensitivity.append(cv_sensitivity)
        repeat_specificity.append(cv_specificity)
        
    print("%.2f%% (+/- %.2f%%)" % (np.mean(repeat_accuracy)*100, np.std(repeat_accuracy)*100))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(repeat_sensitivity)*100, np.std(repeat_sensitivity)*100))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(repeat_specificity)*100, np.std(repeat_specificity)*100))
    
    return repeat_y_pred_proba, repeat_accuracy

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
def plot_roc_curve(repeat_y_true,repeat_y_pred_proba):
    repeat_auc=[]
    for i in range(len(repeat_y_true)):
        auc = roc_auc_score(np.array(list(np.concatenate(repeat_y_true[0]))), np.array(list(np.concatenate(repeat_y_pred_proba[i]))))
        print('AUC: %.3f' % auc)
        repeat_auc.append(auc)

        # calculate roc curve
        fpr, tpr, thresholds = roc_curve(np.array(list(np.concatenate(repeat_y_true[0]))), np.array(list(np.concatenate(repeat_y_pred_proba[i]))))
        # plot no skill
        pyplot.plot([0, 1], [0, 1], linestyle='--')
        # plot the roc curve for the model
        pyplot.plot(fpr, tpr, marker='.')
        # show the plot
        pyplot.show()
    print("%.3f (+/- %.3f)" % (np.mean(repeat_auc), np.std(repeat_auc)))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
def plot_roc_curve_average(repeat_y_true,repeat_y_pred_proba):

    auc = roc_auc_score(np.array(list(np.concatenate(np.mean(np.array(repeat_y_true),axis=0)))), np.array(list(np.concatenate(np.mean(np.array(repeat_y_pred_proba),axis=0)))))
    print('AUC: %.3f' % auc)

    # calculate roc curve
    fpr, tpr, thresholds = roc_curve(np.array(list(np.concatenate(np.mean(np.array(repeat_y_true),axis=0)))), np.array(list(np.concatenate(np.mean(np.array(repeat_y_pred_proba),axis=0)))))
    # plot no skill
    pyplot.plot([0, 1], [0, 1], linestyle='--')
    # plot the roc curve for the model
    pyplot.plot(fpr, tpr, marker='.')
    # show the plot
    pyplot.show()
